In [2]:
!pip install contractions

     |████████████████████████████████| 321 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 283 kB 8.9 MB/s eta 0:00:01
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=102915 sha256=604e1655e23d97ff3c6a0ed6e1f5e12d1e177873fb6fb5a01d02efab700a5f8b
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [3]:
!pip install unidecode

In [4]:
debug = True

In [5]:
import numpy as np
import pandas as pd
import os
import zipfile
import string
import re
import time
import sys
import random
import multiprocessing
import more_itertools


import nltk
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from IPython.display import clear_output

In [6]:
sys.setrecursionlimit(10**8)

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset,Sampler
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt 

from sklearn.metrics import f1_score

import transformers
from transformers import AutoConfig
from transformers import (
    get_cosine_schedule_with_warmup, 
    get_cosine_with_hard_restarts_schedule_with_warmup,
    get_linear_schedule_with_warmup
)
from transformers import AdamW
from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import MODEL_FOR_SEQUENCE_CLASSIFICATION_MAPPING
from IPython.display import clear_output
from tqdm import tqdm, trange


import random
import unidecode
import contractions
from sklearn.metrics import multilabel_confusion_matrix,roc_auc_score,recall_score
import warnings
warnings.simplefilter('ignore')

scaler = torch.cuda.amp.GradScaler() # GPUでの高速化。

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # cpuがgpuかを自動判断
device

device(type='cuda')

In [8]:
from tqdm._tqdm_notebook import tqdm_notebook as tq
tq.pandas()

In [9]:

SEED = 508

def random_seed(SEED):
    
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True

random_seed(SEED)

In [10]:
BASE_PATH = '../input/jigsaw-toxic-comment-classification-challenge'
unzip = zipfile.ZipFile('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
unzip.extractall()
unzip = zipfile.ZipFile('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
unzip.extractall()
unzip = zipfile.ZipFile('../input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
unzip.extractall()
unzip = zipfile.ZipFile('../input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')
unzip.extractall()

In [11]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
train.head(3)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0


In [12]:
train['comment_text'] = train['comment_text'].progress_apply(lambda x: x.replace("\n"," "))
test['comment_text'] = test['comment_text'].progress_apply(lambda x: x.replace("\n"," "))

  0%|          | 0/159571 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

In [13]:
train['comment_text'] = train['comment_text'].fillna(" ")

In [14]:

test['comment_text'] = test['comment_text'].fillna(" ")

In [15]:
def clean_text(text):
  
    text = unidecode.unidecode(text)
    text = contractions.fix(text)
    exclist = string.punctuation 
    table_ = str.maketrans('', '', exclist)
    text = text.translate(table_)
    text = text.replace("'","")
    
    return text

In [16]:
%%time
train['comment_text'] = train['comment_text'].progress_apply(clean_text)
test['comment_text'] = test["comment_text"].progress_apply(clean_text)

  0%|          | 0/159571 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

CPU times: user 24.9 s, sys: 183 ms, total: 25.1 s
Wall time: 25.1 s


In [17]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation Why the edits made under my userna...,0,0,0,0,0,0
1,000103f0d9cfb60f,Daww He matches this background colour I am se...,0,0,0,0,0,0
2,000113f07ec002fd,Hey man I am really not trying to edit war it ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,More I cannot make any real suggestions on im...,0,0,0,0,0,0
4,0001d958c54c6e35,You sir are my hero Any chance you remember wh...,0,0,0,0,0,0


In [18]:
train['comment_text'] = train['comment_text'].progress_apply(lambda x: x[:510])
test['comment_text'] = test['comment_text'].progress_apply(lambda x: x[:450])

  0%|          | 0/159571 [00:00<?, ?it/s]

  0%|          | 0/153164 [00:00<?, ?it/s]

In [19]:
max_len = 500

In [26]:
tokenizer = transformers.AutoTokenizer.from_pretrained('bert-base-uncased')
clear_output()

In [21]:
class BERTDataSet(Dataset):
    
    def __init__(self, sentences, targets):
        
        self.sentences = sentences
        self.targets = targets
        
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self,idx):
        
        sentence = self.sentences[idx]
        
        bert_sens = tokenizer.encode_plus(
                            sentence,
                            add_special_tokens = True,
                            max_length = 300,
                            pad_to_max_length = True,
                            truncation=True,
                            return_attention_mask =True)
        
        ids = torch.tensor(bert_sens['input_ids'], dtype=torch.long)
        mask = torch.tensor(bert_sens['attention_mask'], dtype=torch.long)
#         token_type_ids = torch.tensor(bert_sens['token_type_ids'], dtype=torch.long)
        
        target = torch.tensor(self.targets.iloc[idx].values, dtype=torch.float)
        
        return {
            
            'ids':ids,
            'mask':mask,
#             'token_type_ids':token_type_ids,
            'targets': target
        }
    

In [22]:
import gc
torch.cuda.empty_cache()
gc.collect()

134

In [23]:
# pos_weight = torch.tensor([0.904,0.9899,0.947,0.997,0.950,0.991]).to(device)
def loss_fn(output,target):
    return nn.BCEWithLogitsLoss()(output,target)

In [ ]:
def training(
    dataloader,
    model,
    optimizer,
    scheduler
):
    model.train()
    torch.backends.cudnn.benchmark = True
    
    allpreds = []
    alltargets = []
    
    for a in tqdm_notebook(train_dataloader):

        losses = []

        optimizer.zero_grad()

        with torch.cuda.amp.autocast():

            ids = a["ids"].to(device,non_blocking=True)
            mask = a["mask"].to(device,non_blocking=True)
#             tokentype = a["token_type_ids"].to(device,non_blocking=True)
            logits = model(ids, mask)
            targets = a["targets"].to(device,non_blocking=True)
            loss = loss_fn(logits, targets)

            # For scoring
            losses.append(loss.item())
            allpreds.append(logits.detach().cpu().numpy())
            alltargets.append(targets.detach().squeeze(-1).cpu().numpy())

        scaler.scale(loss).backward() # backwards of loss
        scaler.step(optimizer) # Update optimizer
        scaler.update() # scaler update

        scheduler.step() # Update learning rate schedule

        # Combine dataloader minutes

    allpreds = np.concatenate(allpreds)
    alltargets = np.concatenate(alltargets).astype(int)

    # I don't use loss, but I collect it

    losses = np.mean(losses)
    
    allpreds = (allpreds >0.5).astype(int)
    alltargets = (alltargets > 0.5).astype(int)
    
    try:
        print(f"Confusion matrix : \n{multilabel_confusion_matrix(alltargets,allpreds)}")
    except TypeError as err:
        print("Type error: {0}".format(err))
    except ValueError as err:
        print("Type error: {0}".format(err))

    try:
        print(f"recall score: \n{recall_score(alltargets,allpreds,average='macro')}")
    except TypeError as err:
        print("Type error: {0}".format(err))
    except ValueError as err:
        print("Type error: {0}".format(err))

    try:
        print(f"roc_auc score: \n{roc_auc_score(alltargets,allpreds,average='macro')}")
    except TypeError as err:
        print("Type error: {0}".format(err))
    except ValueError as err:
        print("Type error: {0}".format(err))
        # Score with rmse
        # Score with rmse
    train_bce_loss = roc_auc_score(alltargets,allpreds,average="macro")

    return losses,train_bce_loss


In [ ]:
def validation(
    dataloader,
    model,
):
    model.eval()
    
    allpreds = []
    alltargets = []
    
    for a in tqdm_notebook(valid_dataloader):
        
        losses = []
        
        with torch.no_grad():
            
            ids = a["ids"].to(device, non_blocking=True)
            mask = a["mask"].to(device, non_blocking=True)
#             tokentype = a["token_type_ids"].to(device, non_blocking=True)
            
            logits = model(ids, mask)
            
            targets = a["targets"].to(device, non_blocking=True)
            loss = loss_fn(logits, targets)

            # For scoring
            losses.append(loss.item())
            allpreds.append(logits.detach().cpu().numpy())
            alltargets.append(targets.detach().squeeze(-1).cpu().numpy())

    allpreds = np.concatenate(allpreds)
    alltargets = np.concatenate(alltargets)

    # I don't use loss, but I collect it

    losses = np.mean(losses)
    
#     alltargets = [int(a) for a in alltargets]
    allpreds = (allpreds >0.5).astype(int)
    alltargets = (alltargets > 0.5).astype(int)
    
    try:
        print(f"Confusion matrix : \n{multilabel_confusion_matrix(alltargets,allpreds)}")
    except TypeError as err:
        print("Type error: {0}".format(err))
    except ValueError as err:
        print("Type error: {0}".format(err))
        
    try:
        print(f"recall score: \n{recall_score(alltargets,allpreds,average='macro')}")
    except TypeError as err:
        print("Type error: {0}".format(err))
    except ValueError as err:
        print("Type error: {0}".format(err))
    
    try:
        print(f"roc_auc score: \n{roc_auc_score(alltargets,allpreds,average='macro')}")
    except TypeError as err:
        print("Type error: {0}".format(err))
    except ValueError as err:
        print("Type error: {0}".format(err))
    # Score with rmse
    valid_bce_loss = roc_auc_score(alltargets,allpreds,average="macro")

    return allpreds, losses, valid_bce_loss


In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()

def loss_fn(output,target):
    return nn.BCEWithLogitsLoss()(output,target)

In [24]:
# initializing the data
from transformers import AdamW
from transformers import get_linear_schedule_with_warmup
valid_batch =64
train_batch=32
epochs = 10
LR=5e-6

train = train.sort_values("toxic").reset_index(drop=True)
train["kfold"] = train.index % 5

p_train = train[train["kfold"]!=1].reset_index(drop=True)
p_valid = train[train["kfold"]==1].reset_index(drop=True)

train_dataset = BERTDataSet(p_train["comment_text"], p_train[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]])
valid_dataset = BERTDataSet(p_valid["comment_text"], p_valid[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]])

train_dataloader = DataLoader(train_dataset, batch_size=train_batch, shuffle=True,num_workers=4,pin_memory=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=valid_batch, shuffle=False, num_workers=4,pin_memory=True)

model = transformers.BertForSequenceClassification.from_pretrained("./bert-base-uncased",num_labels=6)
clear_output()

model.to(device)
LR=5e-6
optimizer = AdamW(model.parameters(), LR,betas=(0.9, 0.999), weight_decay=1e-2) # AdamW optimizer

train_steps = int(len(p_train)/train_batch*epochs)

num_steps = int(train_steps*0.1)

scheduler = get_linear_schedule_with_warmup(optimizer, num_steps, train_steps)

In [ ]:
trainlosses = []
vallosses = []
bestscore = None

trainscores = []
validscores = []

for epoch in tqdm_notebook(range(epochs)):
    
    print("---------------------" + str(epoch) + "start-------------------")
    
    trainloss,trainscore = training(train_dataloader, model, optimizer, scheduler)
    
    trainlosses.append(trainloss)
    trainscores.append(trainscore)
    
    print("traiscore is " + str(trainscores))
    print("trainloss is " + str(trainloss))
    
    preds, validloss, valscore = validation(valid_dataloader, model)
    
    vallosses.append(validloss)
    validscores.append(valscore)
    
    print("validscore is " + str(valscore))
    print("validloss is " + str(vallosses))
    
    if bestscore is None:
        bestscore = valscore
        
        print("Save first model")
        
        state = {
                        'state_dict' : model.state_dict(),
                        'optimizer_dict' : optimizer.state_dict(),
                        'bestscore': bestscore
                    }
        
        torch.save(state, "model_bce.pth")
        
    elif bestscore < valscore:
        bestscore = valscore
        
        print("found better point")
        
        state = {
                        'state_dict' : model.state_dict(),
                        'optimizer_dict' : optimizer.state_dict(),
                        'bestscore': bestscore
                    }
        
        torch.save(state,"model_bce.pth")
        
    else:
        pass

In [27]:
class BERTinfDataSet(Dataset):
    
    def __init__(self,sentences):
        
        self.sentences = sentences
       
        
    def __len__(self):
        
        return len(self.sentences)
    
    def __getitem__(self,idx):
        
        sentence = self.sentences[idx]
        
        
        bert_sens = tokenizer.encode_plus(
                                sentence,
                                add_special_tokens = True, # [CLS],[SEP]
                                max_length = 314,
                                pad_to_max_length = True, # add padding to blank
                                truncation=True)

        ids = torch.tensor(bert_sens['input_ids'], dtype=torch.long)
        mask = torch.tensor(bert_sens['attention_mask'], dtype=torch.long)
        token_type_ids = torch.tensor(bert_sens['token_type_ids'], dtype=torch.long)
     
        
    
        
        return {
                'ids': ids,
                'mask': mask,
                'token_type_ids': token_type_ids,
                
            }

In [ ]:
test_dataset = BERTinfDataSet(test["comment_text"])

In [ ]:
test_batch = 32

In [ ]:
test_dataloader = DataLoader(test_dataset,batch_size=test_batch,shuffle = False,num_workers=4,pin_memory=True)

In [ ]:
model = transformers.BertForSequenceClassification.from_pretrained('./bert-base-uncased',num_labels=6)

In [ ]:
pthes = [os.path.join("./",s) for s in os.listdir("./") if ".pth" in s]
pthes

In [ ]:
def predicting(
    test_dataloader,
    model,
    pthes
    
):

    allpreds = []
    
    for pth in pthes:
        
        state = torch.load(pth)
        
        model.load_state_dict(state["state_dict"])
        model.to(device)
        model.eval()
    
    
        preds = []
        allvalloss=0

        with torch.no_grad():


            for a in tqdm_notebook(test_dataloader):



                ids = a["ids"].to(device)
                mask = a["mask"].to(device)
                tokentype = a["token_type_ids"].to(device)

               # output = model(ids,mask,tokentype)
                output = model(ids,mask)

                output = output["logits"]


                preds.append(output.cpu().numpy())

            preds = np.concatenate(preds)
            
            allpreds.append(preds)

    return allpreds


In [ ]:
sequence = train['comment_text'][0]

tokenized_sequence = tokenizer.tokenize(sequence)

tokenized_sequence

In [ ]:
allpreds = predicting(test_dataloader,model,pthes)

In [ ]:
prediction= np.concatenate(allpreds)

In [ ]:
findf = pd.DataFrame(prediction)
subm = pd.read_csv("sample_submission.csv")
subm[["toxic","severe_toxic","obscene","threat","insult","identity_hate"]] = findf